In [23]:
# extract features via librosa
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

audio = '/Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_55_333231/01.wav'
y, sr = librosa.load(audio)

# get stft of audio
stft = librosa.stft(y)

# extract mfcc
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

# extract mel
mel = librosa.feature.melspectrogram(y=y, sr=sr)

# extract contrast
contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

# extract spectral centroid
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)

# extract spectral bandwidth
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)

# extract spectral rolloff
spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.99)
spec_rolloff_min = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.01)

# extract pitch(f0) from time series
f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                             fmin=librosa.note_to_hz('C2'),
                                             fmax=librosa.note_to_hz('C7'))
f0 = f0[np.newaxis, :]
voiced_flag = voiced_flag[np.newaxis, :]
voiced_probs = voiced_probs[np.newaxis, :]

times_f0 = librosa.times_like(f0)
# extract zero crossing rate
zcr = librosa.feature.zero_crossing_rate(y=y)

# extract flatness
flatness = librosa.feature.spectral_flatness(y=y)


In [24]:
# feature extractor class
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler


class FeatureExtractor:
    def __init__(self, sr=22050):
        self.sr = sr

    def extract_features(self, audio_path):
        y, sr = librosa.load(audio_path, sr=self.sr)
        
        # # extract mfcc
        # mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        #
        # # extract mel
        # mel = librosa.feature.melspectrogram(y=y, sr=sr)
        #
        # # extract contrast
        # contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        #
        # # extract spectral centroid
        # spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        #
        # # extract spectral bandwidth
        # spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        #
        # # extract spectral rolloff
        # spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.99)
        # spec_rolloff_min = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.01)

        # extract pitch(f0) from time series
        f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                                     fmin=librosa.note_to_hz('C2'),
                                                     fmax=librosa.note_to_hz('C7'),
                                                     fill_na=0.0)
        f0 = f0[np.newaxis, :]
        voiced_flag = voiced_flag[np.newaxis, :]
        voiced_probs = voiced_probs[np.newaxis, :]
        
        # # extract zero crossing rate
        # zcr = librosa.feature.zero_crossing_rate(y=y)
        #
        # # extract flatness
        # flatness = librosa.feature.spectral_flatness(y=y)
        
        # concatenate all features
        # features = np.concatenate((mfcc, mel, contrast, spec_cent, spec_bw, spec_rolloff, spec_rolloff_min, f0, voiced_flag, voiced_probs, zcr, flatness), axis=0)
        features = np.concatenate((f0, voiced_probs), axis=0)
        # Aggregate features
        # features = np.nan_to_num(features, nan=0.0)
        # features[~np.isfinite(features)] = 0
        features = np.vstack((np.mean(features, axis=1))).flatten()
        
        return features

In [25]:
import numpy as np
import librosa
import os
import re
from pprint import pprint
# from natsort import natsorted
from tqdm import tqdm
import glob
import multiprocessing

base_folder_path = '/Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples'

# features_folder_path = f'{base_folder_path}-features'
# if not os.path.exists(features_folder_path):
#     os.makedirs(features_folder_path)

# get all the folders under the base folder
# wav_files = natsorted(glob.glob(os.path.join(base_folder_path, '**/*.wav'), recursive=True))
wav_files = glob.glob(os.path.join(base_folder_path, '**/*.wav'), recursive=True)


import multiprocess
import numpy as np

# wav_files_no_0_labels = [
#     wav_file for wav_file in wav_files
#     if not (int(re.findall(r'\d+', wav_file)[-2][-3]) == 0 or int(re.findall(r'\d+', wav_file)[-2][-1]) == 0)]

labels_G = []
labels_T = []
labels_R = []
features = []

def process_file(wav_file):
    # 从文件路径中提取数字
    digits = re.findall(r'\d+', wav_file)[-2]
    try:
        G = int(digits[-3])
        T = int(digits[-2])
        R = int(digits[-1])

        # 提取特征
        extractor = FeatureExtractor()
        feature = extractor.extract_features(audio_path=wav_file)

        # feature_file_path = os.path.join(features_folder_path, os.path.basename(wav_file) + '.npy')
        # np.save(feature_file_path, feature)  # 保存特征到文件

        return (G, T, R, feature, f'Processed {wav_file}_G{G}_T{T}_R{R}')
    except Exception as e:
        return None, None, None, None, f"Error processing {wav_file}: {e}"

with multiprocess.Pool() as pool:
    results = list(tqdm(pool.imap(process_file, wav_files), total=len(wav_files)))

for G, T, R, feature, message in results:
    if feature is not None:
        labels_G.append(G)
        labels_T.append(T)
        labels_R.append(R)
        features.append(feature)
        print(message)
    else:
        print(message)

features = np.array(features)
labels_G = np.array(labels_G)
labels_T = np.array(labels_T)
labels_R = np.array(labels_R)



100%|██████████| 167/167 [00:38<00:00,  4.36it/s]

Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_120_333423/01.wav_G4_T2_R3
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_25_333136/01.wav_G1_T3_R6
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_99_333343/01.wav_G3_T4_R3
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_44_333214/01.wav_G2_T1_R4
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_90_333331/01.wav_G3_T3_R1
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_102_333346/01.wav_G3_T4_R6
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_37_333154/01.wav_G1_T5_R4
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_86_333324/01.wav_G3_T2_R4
Processed /Users/yizhong/Documents/projects/artivoice/GTR-145-Demo/samples/gtr_132_333441/01.wav_G4_T4_R1
Processed /Users/yizhong/Documents/projects/artivoic

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(features, labels_R, test_size=0.2, random_state=32)
X_train

array([[1.19042818e+01, 1.63108194e-02],
       [1.98867368e+02, 3.83507428e-01],
       [5.25858889e+01, 1.26463919e-01],
       [1.29870532e+02, 1.98621011e-01],
       [2.29517300e+02, 3.99050978e-01],
       [2.39572837e+02, 3.99242580e-01],
       [2.77680959e+02, 3.29907471e-01],
       [1.74793736e+02, 2.57452263e-01],
       [2.10792538e+02, 3.19205818e-01],
       [1.97007048e+02, 3.73292308e-01],
       [1.92216158e+02, 1.71895762e-01],
       [1.93335445e+02, 3.69082473e-01],
       [2.53164762e+02, 4.15919463e-01],
       [2.40263949e+02, 2.71724125e-01],
       [5.80040974e+01, 1.21660869e-01],
       [1.37222942e+01, 1.60428473e-02],
       [1.59574405e+01, 1.32639184e-02],
       [2.21899580e+02, 2.25700556e-01],
       [2.53558367e+02, 5.07045610e-01],
       [2.82972309e+02, 5.50859063e-01],
       [4.96512221e+01, 9.02662790e-02],
       [1.11460955e+02, 2.34361814e-01],
       [2.04906709e+02, 4.40533338e-01],
       [2.26673051e+02, 2.76524656e-01],
       [2.246634

In [27]:
# minmax the features
# scaler = MinMaxScaler()
#
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# check mean and std of the scaled features
# print(f'Mean of the scaled features: {np.mean(X_train_scaled)}')
# print(f'Std of the scaled features: {np.std(X_train_scaled)}')

In [39]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# t
clf = make_pipeline(SVC(C=0.1, gamma=0.001, kernel='rbf'))
# clf = make_pipeline(SVC())
clf.fit(X_train, y_train)
# clf.fit(X_train, y_train)

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# 在训练集上评估模型
y_train_pred = clf.predict(X_train)
print("Train set report:")
print(classification_report(y_train, y_train_pred))

# 在测试集上进行预测
y_test_pred = clf.predict(X_test)

# 测试集评估
print("Test set report:")
print(classification_report(y_test, y_test_pred))


Train set report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.67      1.00      0.80        18
           2       0.00      0.00      0.00        15
           3       0.00      0.00      0.00        14
           4       0.27      0.61      0.37        18
           5       0.00      0.00      0.00        16
           6       0.26      0.71      0.38        17
           7       0.00      0.00      0.00        16

    accuracy                           0.45       132
   macro avg       0.27      0.41      0.32       132
weighted avg       0.30      0.45      0.35       132

Test set report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.50      1.00      0.67         2
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00

/Users/yizhong/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yizhong/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yizhong/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yizhong/minicon

In [1]:

# save the report as a text file
with open('classification_report.txt', 'w') as f:
    f.write(f"Train set report:\n{classification_report(y_train, y_train_pred)}\n\n")
    f.write(f"Test set report:\n{classification_report(y_test, y_test_pred)}\n\n")

# plot loss curve
# plt.plot(clf[-1].loss_curve_)

NameError: name 'classification_report' is not defined

In [ ]:
# from sklearn.model_selection import GridSearchCV
#
# param_grid = {
#     'C': [0.1, 1, 10, 100, 1000],
#     'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#     'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
# }
#
# grid_search = GridSearchCV(SVC(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)
#
# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best score: {grid_search.best_score_}")

In [ ]:
# print all features shape
print('y: ', y.shape)
print('stft: ', stft.shape)
print('mfcc: ', mfcc.shape)
print('mel: ', mel.shape)
print('contrast: ', contrast.shape)
print('spectral centroid: ', spec_cent.shape)
print('spectral bandwidth: ', spec_bw.shape)
print('spectral rolloff: ', spec_rolloff.shape)
print('spectral rolloff min: ', spec_rolloff_min.shape)
print('f0: ', f0.shape)
print('voiced flag: ', voiced_flag.shape)
print('voiced probs: ', voiced_probs.shape) 
print('zero crossing rate: ', zcr.shape)
print('flatness: ', flatness.shape)

In [ ]:
# plot all features

# mfcc
plt.figure(figsize=(20, 10))
plt.subplot(3, 3, 7)  # Change from (2, 4, 1) to (3, 3, 1)
librosa.feature.mfcc(S=librosa.power_to_db(mel))

librosa.display.specshow(mfcc, x_axis='time')

plt.title('MFCC')
plt.colorbar()
plt.tight_layout()

# mel spectrogram
plt.subplot(3, 3, 2)  # Change from (2, 4, 2) to (3, 3, 2)
ax = plt.gca()
S_dB = librosa.power_to_db(mel, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time',
                         y_axis='mel', sr=sr,
                         fmax=sr/2, ax=ax)
plt.title('Mel')
plt.tight_layout()

# contrast
plt.subplot(3, 3, 3)  # Change from (2, 4, 3) to (3, 3, 3)
librosa.display.specshow(contrast, x_axis='time')
plt.title('Contrast')
# plt.colorbar()
plt.tight_layout()

# spectral centroid
plt.subplot(3, 3, 4)  # Change from (2, 4, 4) to (3, 3, 4)
ax = plt.gca() # get current axis
S, phase = librosa.magphase(librosa.stft(y=y))
times = librosa.times_like(spec_cent)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                         y_axis='log', x_axis='time', ax=ax)
ax.plot(times, spec_cent[0], label='Spectral centroid', color='w')
plt.title('Spectral Centroid')
# plt.colorbar()
plt.tight_layout()

# spectral bandwidth
plt.subplot(3, 3, 5)  # Change from (2, 4, 5) to (3, 3, 5)
ax = plt.gca()  # 获取当前的Axes对象

# band width
times = librosa.times_like(spec_bw)
centroid = librosa.feature.spectral_centroid(S=mel)
ax.semilogy(times, spec_bw[0], label='Spectral bandwidth', color='b')  
ax.set(ylabel='Hz', xticks=[], xlim=[times.min(), times.max()])
ax.legend()
ax.label_outer()

librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                         y_axis='log', x_axis='time', ax=ax)

ax.fill_between(times, np.maximum(0, centroid[0] - spec_bw[0]),
                np.minimum(centroid[0] + spec_bw[0], sr/2),
                alpha=0.5, label='Centroid +- bandwidth', color='r')  
ax.plot(times, centroid[0], label='Spectral centroid', color='w')
ax.legend(loc='lower right')
plt.title('Spectral Bandwidth')

# spectral rolloff
plt.subplot(3, 3, 6)  # Change from (2, 4, 6) to (3, 3, 6)
ax = plt.gca()  # 获取当前的Axes对象
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                         y_axis='log', x_axis='time', ax=ax)
ax.plot(librosa.times_like(spec_rolloff), spec_rolloff[0], label='Roll-off frequency (0.99)', color='c')
ax.plot(librosa.times_like(spec_rolloff), spec_rolloff_min[0], color='m', label='Roll-off frequency (0.01)')
ax.legend(loc='lower right')
plt.title('Spectral Rolloff')

# pitch
plt.subplot(3, 3, 1)  # Change from (2, 4, 7) to (3, 3, 7)
ax = plt.gca()
times_f0 = librosa.times_like(f0)
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)S
librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
ax.set(title='pYIN fundamental frequency estimation')
ax.plot(times_f0, f0, label='f0', color='cyan', linewidth=3)
ax.legend(loc='upper right')

# zero crossing rate
plt.subplot(3, 3, 8)  # Change from (2, 4, 8) to (3, 3, 8)
plt.plot(zcr[0])
plt.title('Zero Crossing Rate')
plt.tight_layout()

# flatness
plt.subplot(3, 3, 9)  # Add new subplot (3, 3, 9)
plt.plot(flatness[0])
plt.title('Spectral Flatness')
plt.tight_layout()

plt.show()